### load the imports and intel extension

In [1]:
import pandas as pd
import torch
import numpy as np
############# code changes ###############
import intel_extension_for_pytorch as ipex
############# code changes ###############

/home/u136698/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### load the files

In [4]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

In [5]:
count = 0
stop = False
for _, row in train_df.iterrows():
    if row["Answer"] not in row["Story"]:
        count += 1
        if not stop:
            print(row["Question"], row["Answer"])
            print(row["Story"])
            print("--------------------------")
    if count == 2:
        stop = True
print(count/len(train_df))

Is it a large country? No
Malawi (, or ; or [maláwi]), officially the Republic of Malawi, is a landlocked country in southeast Africa that was formerly known as Nyasaland. It is bordered by Zambia to the northwest, Tanzania to the northeast, and Mozambique on the east, south and west. Malawi is over with an estimated population of 16,777,547 (July 2013 est.). Its capital is Lilongwe, which is also Malawi's largest city; the second largest is Blantyre, the third is Mzuzu and the fourth largest is its old capital Zomba. The name Malawi comes from the Maravi, an old name of the Nyanja people that inhabit the area. The country is also nicknamed "The Warm Heart of Africa". 

Malawi is among the smallest countries in Africa. Lake Malawi takes up about a third of Malawi's area. 

The area of Africa now known as Malawi was settled by migrating Bantu groups around the 10th century. Centuries later in 1891 the area was colonised by the British. In 1953 Malawi, then known as Nyasaland, a protecto

In [6]:
# sjrhuschlee/flan-t5-large-squad2
# mrm8488/t5-base-finetuned-squadv2

### model to finetune

In [7]:
model_checkpoint = "mrm8488/t5-base-finetuned-squadv2"

In [8]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [9]:
train_data = train_df.iloc[:60000].reset_index(drop=True)
valid_data = train_df.iloc[60000:].reset_index(drop=True)

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


### preprocess the data

In [11]:
train_data["input"] = train_data.apply(lambda x: f'question: {x["Question"]}  context: {x["Story"]}', axis=1)
valid_data["input"] = valid_data.apply(lambda x: f'question: {x["Question"]}  context: {x["Story"]}', axis=1)

In [12]:
from datasets import load_dataset, load_metric, Dataset

In [13]:
# Load datasets
train_dataset = Dataset.from_pandas(train_data[["input", "Answer"]])
valid_dataset = Dataset.from_pandas(valid_data[["input", "Answer"]])

In [14]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = examples["input"]
    targets = examples["Answer"]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
train_tokenized_datasets = train_dataset.map(preprocess_function, batched=True)
valid_tokenized_datasets = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6611 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


## optimize the model

In [17]:
#################### code changes #################
model = model.to("xpu")
model = ipex.optimize(model)
#################### code changes #################

/home/u136698/.local/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:611: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/home/u136698/.local/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:618: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(


In [18]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 222903552
all model parameters: 222903552
percentage of trainable model parameters: 100.00%


In [19]:
batch_size = 6

In [20]:
model.generation_config.max_length = max_target_length

In [21]:
args = Seq2SeqTrainingArguments(
    f"new_model",
    evaluation_strategy = "steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    eval_steps=10,
    save_steps=10,
    report_to="none",
    load_best_model_at_end=True
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

### adding eval metric and bleu score

In [23]:
import numpy as np
from sklearn.metrics import f1_score

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Calculate Exact Match (EM) and F1 score
    em_score = 0
    f1_score_value = 0
    for pred, label in zip(decoded_preds, decoded_labels):
        em_score += int(pred == label)
        f1_score_value += f1_score([label], [pred], average='macro')

    em_score = em_score / len(decoded_preds)  # EM is a ratio
    f1_score_value = f1_score_value / len(decoded_preds)  # F1 score is an average

    result = {
        "exact_match": round(em_score, 4),
        "f1_score": round(f1_score_value, 4)
    }

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=valid_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/u136698/.local/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


### train and save the model

In [ ]:
trainer.train()

In [26]:
trainer.save_model("../T5-trained")

In [27]:
from neural_compressor.config import PostTrainingQuantConfig
from neural_compressor import quantization

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## quantize the trained model

In [28]:
conf = PostTrainingQuantConfig(
    device="gpu",
    backend="itex",
    approach="dynamic"
)

In [29]:
calib_dataloader = trainer.get_eval_dataloader()
conf.use_bf16 = False

In [30]:
q_model = quantization.fit(
    model,
    conf,
    calib_dataloader=calib_dataloader
)

2023-10-20 10:45:14 [INFO] Start auto tuning.
2023-10-20 10:45:14 [INFO] Quantize model without tuning!
2023-10-20 10:45:14 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.
2023-10-20 10:45:14 [INFO] Adaptor has 5 recipes.
2023-10-20 10:45:14 [INFO] 0 recipes specified by user.
2023-10-20 10:45:14 [INFO] 3 recipes require future tuning.
2023-10-20 10:45:14 [INFO] *** Initialize auto tuning
2023-10-20 10:45:14 [INFO] {
2023-10-20 10:45:14 [INFO]     'PostTrainingQuantConfig': {
2023-10-20 10:45:14 [INFO]         'AccuracyCriterion': {
2023-10-20 10:45:14 [INFO]             'criterion': 'relative',
2023-10-20 10:45:14 [INFO]             'higher_is_better': True,
2023-10-20 10:45:14 [INFO]             'tolerable_loss': 0.01,
2023-10-20 10:45:14 [INFO]             'absolute': None,
2023-10-20 10:45:14 [INFO]     

### save the quantized model

In [31]:
q_model.save("../quantized_model")

2023-10-20 10:45:24 [INFO] Save config file and weights of quantized model to /home/u136698/Training/oneAPI_final/quantized_model.
